# Python Fortgeschritten: XML Verarbeitung
## Tag 4 - Notebook 23
***
In diesem Notebook wird behandelt:
- xml.etree.ElementTree
- DOM
- SAX
- XML-Parsing
***


## 1 XML Verarbeitung in Python

XML (eXtensible Markup Language) ist ein Markup-Format zur Strukturierung von Daten. Python bietet mehrere Module zur XML-Verarbeitung, wobei `xml.etree.ElementTree` die einfachste und am häufigsten verwendete API ist.

### Was ist XML?

XML ist ein textbasiertes Format zur Darstellung hierarchischer Daten. Es verwendet Tags (Elemente), Attribute und verschachtelte Strukturen, ähnlich wie HTML, aber mit benutzerdefinierten Tags.

### Wann und warum verwenden?

XML sollte verwendet werden, wenn:
- **Strukturierte Konfiguration**: Konfigurationsdateien mit komplexer Struktur benötigt werden. XML eignet sich gut für strukturierte, hierarchische Daten.
- **Daten-Austausch**: Daten zwischen verschiedenen Systemen ausgetauscht werden sollen. XML ist textbasiert, plattformunabhängig und ein etablierter Standard mit vielen Tools und Bibliotheken.
- **Legacy-Systeme**: Integration mit Systemen, die XML verwenden
- **Dokumenten-Markup**: Dokumente mit komplexer Struktur (z.B. DocBook, XHTML) benötigt werden. Die Struktur der Daten ist im XML selbst enthalten (selbstbeschreibend).

### XML-Parsing-Ansätze

Python bietet verschiedene Ansätze zur XML-Verarbeitung:

1. **ElementTree (DOM-ähnlich)**: Einfach zu verwenden, lädt das gesamte XML in den Speicher
2. **SAX (Streaming)**: Event-basiert, verarbeitet XML während des Lesens
3. **lxml**: Erweiterte Bibliothek mit mehr Features (nicht Teil der Standardbibliothek)

### Vorteile von ElementTree

- **Einfache API**: Intuitive und leicht zu erlernende API
- **Pythonisch**: Fühlt sich natürlich in Python an
- **Teil der Standardbibliothek**: Keine zusätzlichen Installationen nötig
- **Ausreichend für die meisten Fälle**: Deckt die meisten XML-Verarbeitungsaufgaben ab

### Einschränkungen

- **Speicher**: Lädt das gesamte XML in den Speicher (bei sehr großen Dateien problematisch)
- **Features**: Weniger Features als lxml
- **Performance**: Kann bei sehr großen XML-Dateien langsam sein


In [ ]:
import xml.etree.ElementTree as ET

# XML erstellen
root = ET.Element("root")
child = ET.SubElement(root, "child")
child.text = "Text"

# XML parsen
# tree = ET.parse('file.xml')
# root = tree.getroot()


## 2 ElementTree API im Detail

`xml.etree.ElementTree` bietet eine einfache API für XML-Verarbeitung. Die Hauptkomponenten sind:

- **Element**: Repräsentiert ein XML-Element (Tag)
- **ElementTree**: Repräsentiert das gesamte XML-Dokument
- **SubElement**: Hilfsfunktion zum Erstellen von Unterelementen

### DOM vs. SAX

**DOM (Document Object Model)**:
- Lädt das gesamte XML in den Speicher
- Ermöglicht zufälligen Zugriff auf Elemente
- Einfacher zu verwenden
- **ElementTree ist DOM-ähnlich**

**SAX (Simple API for XML)**:
- Verarbeitet XML während des Lesens (Streaming)
- Speicher-effizient für große Dateien
- Event-basiert (Callbacks)
- Komplexer zu verwenden

**pandas (read_xml)**:
- Liest XML direkt in DataFrame
- Nutzt intern **lxml** (falls installiert) oder **ElementTree** als Fallback
- Beide sind **DOM-basiert** (laedt alles in den Speicher)
- Ideal fuer tabellarische Daten und statistische Auswertungen

In [ ]:
# ============================================
# Beispiel DOM (ElementTree) - Laedt alles in den Speicher
# ============================================
import xml.etree.ElementTree as ET

# XML-Datei laden (DOM-aehnlich)
tree = ET.parse("../data/measurements.xml")
root = tree.getroot()

# Zugriff auf Attribute und Elemente
print("=== DOM Beispiel ===")
print(f"Projekt: {root.get('project')}")
print(f"Ort: {root.find('metadata/location').text}")

# Alle Sensoren durchlaufen
for sensor in root.findall("sensor"):
    sensor_type = sensor.get("type")
    readings = sensor.findall("reading")
    print(f"Sensor {sensor.get('id')}: {len(readings)} Messwerte ({sensor_type})")

# ============================================
# Beispiel SAX - Streaming, speichereffizient
# ============================================
import xml.sax
from xml.sax.handler import ContentHandler

class MeasurementHandler(ContentHandler):
    """SAX Handler zum Zaehlen der Messwerte"""
    def __init__(self):
        self.reading_count = 0
        self.current_sensor = None
    
    def startElement(self, name, attrs):
        if name == "sensor":
            self.current_sensor = attrs.get("type")
        elif name == "reading":
            self.reading_count += 1
    
    def endDocument(self):
        print(f"\n=== SAX Beispiel ===")
        print(f"Gesamt: {self.reading_count} Messwerte gefunden")

# SAX Parser ausfuehren
handler = MeasurementHandler()
xml.sax.parse("../data/measurements.xml", handler)

# ============================================
# Beispiel pandas - Nutzt lxml/etree (DOM-basiert)
# ============================================
# pandas.read_xml() verwendet intern lxml (falls installiert) oder
# xml.etree.ElementTree als Fallback. Beide sind DOM-basiert,
# d.h. das gesamte XML wird in den Speicher geladen.

import pandas as pd

# Methode 1: Direkt mit read_xml (fuer einfache, flache Strukturen)
# XPath waehlt die Elemente aus, die zu Zeilen werden sollen
df_readings = pd.read_xml("../data/measurements.xml", xpath="//reading")
print("\n=== pandas Beispiel (read_xml) ===")
print(df_readings.head())

# Methode 2: ElementTree + pandas (fuer komplexe, verschachtelte Strukturen)
# Hier extrahieren wir manuell und behalten Kontext (Sensor-Info)
data = []
for sensor in root.findall("sensor"):
    sensor_id = sensor.get("id")
    sensor_type = sensor.get("type")
    unit = sensor.get("unit")
    for reading in sensor.findall("reading"):
        data.append({
            "sensor_id": sensor_id,
            "type": sensor_type,
            "unit": unit,
            "timestamp": reading.get("timestamp"),
            "value": float(reading.text)
        })

df = pd.DataFrame(data)
print("\n=== pandas + ElementTree (verschachtelt) ===")
print(df)

# Jetzt pandas-Funktionen nutzen: Gruppierung, Statistik
print("\n=== Statistik mit pandas ===")
print(df.groupby("type")["value"].agg(["mean", "min", "max"]).round(2))

## 3 Best Practices

### Best Practices

1. **ElementTree für die meisten Fälle**: `xml.etree.ElementTree` ist für die meisten Anwendungsfälle ausreichend und einfach zu verwenden.

2. **SAX für große Dateien**: Bei sehr großen XML-Dateien (mehrere GB) sollte SAX verwendet werden, um Speicher zu sparen.

3. **Namespaces beachten**: XML-Namespaces können komplex sein - verwenden Sie `{namespace}tag` Syntax in ElementTree.

4. **Fehlerbehandlung**: XML-Parsing kann fehlschlagen - verwenden Sie `try/except` für robuste Fehlerbehandlung.

5. **Sicherheit**: Vorsicht bei XML von unbekannten Quellen - XXE (XML External Entity) Angriffe sind möglich.

### XML vs. Alternativen

**JSON verwenden, wenn**:
- Daten sind einfach strukturiert (keine komplexe Hierarchie)
- Performance ist wichtig (JSON ist schneller)
- Daten werden hauptsächlich von JavaScript/Web-Anwendungen verwendet
- Einfache API-Responses

**YAML verwenden, wenn**:
- Menschenlesbare Konfigurationsdateien benötigt werden
- Kommentare in Konfigurationsdateien wichtig sind
- Einfache, verschachtelte Strukturen

**XML verwenden, wenn**:
- Komplexe, verschachtelte Strukturen benötigt werden
- Legacy-Systeme XML verwenden
- Dokumenten-Markup benötigt wird
- Standardisierte Datenformate (z.B. RSS, Atom)

### Wann NICHT verwenden?

- **Einfache Datenstrukturen**: JSON oder YAML sind einfacher für einfache Daten
- **Performance-kritisch**: JSON ist schneller zu parsen
- **Web-APIs**: JSON ist der Standard für moderne Web-APIs

### Sicherheitsüberlegungen

- **XXE-Angriffe**: XML External Entity Angriffe können Dateien lesen oder Server-Side Request Forgery verursachen
- **Gegenmaßnahmen**: 
  - XML von unbekannten Quellen nicht verarbeiten
  - Entity-Expansion deaktivieren
  - XML-Parser mit Sicherheitsfeatures verwenden (z.B. `defusedxml`)

### Häufige Fehler

- **Vergessene Namespaces**: XML-Namespaces nicht beachten führt zu fehlenden Elementen
- **Speicher-Probleme**: Sehr große XML-Dateien mit ElementTree laden (→ SAX verwenden)
- **Fehlende Fehlerbehandlung**: XML-Parsing-Fehler nicht abfangen
- **Sicherheitslücken**: XML von unbekannten Quellen ohne Validierung verarbeiten


## 4 Aufgaben

### (a) XML programmatisch erstellen - Messdaten-Struktur

Erstelle eine XML-Struktur fuer eine Messreihe mit folgenden Anforderungen:

- Ein Root-Element `experiment` mit Attributen `id` und `date`
- Ein `metadata`-Element mit Unterelementen: `researcher`, `location`, `description`
- Mindestens 3 `measurement`-Elemente mit Attribut `timestamp`
- Jede Messung hat: `value` (mit Attribut `unit`) und `quality` als Unterelemente
- Gib das fertige XML mit `ET.dump()` aus



In [ ]:
# Deine Loesung:
import xml.etree.ElementTree as ET

# TODO: Root-Element erstellen mit Attributen (id, date)

# TODO: Metadata-Element mit Unterelementen hinzufuegen

# TODO: Messungen hinzufuegen mit SubElement

# TODO: XML ausgeben mit ET.dump()


### (b) XML-String parsen - Sensordaten auswerten

Parse den XML-String mit Sensormesswerten aus der naechsten Zelle:

Anforderungen:
- Parse den XML-String mit `ET.fromstring()`
- Gib alle Sensor-IDs und deren Typ aus
- Berechne den Durchschnittswert jedes Sensors
- Zeige nur Messungen mit quality="valid"

In [ ]:
# Deine Loesung:
import xml.etree.ElementTree as ET

xml_string = '''<sensor_data station="Wetterstation-Nord">
    <sensor id="T01" type="temperature" unit="Celsius">
        <reading timestamp="08:00" quality="valid">18.5</reading>
        <reading timestamp="12:00" quality="valid">23.2</reading>
        <reading timestamp="16:00" quality="invalid">-999</reading>
        <reading timestamp="20:00" quality="valid">19.8</reading>
    </sensor>
    <sensor id="H01" type="humidity" unit="Prozent">
        <reading timestamp="08:00" quality="valid">65.2</reading>
        <reading timestamp="12:00" quality="valid">48.7</reading>
        <reading timestamp="16:00" quality="valid">52.1</reading>
        <reading timestamp="20:00" quality="valid">71.3</reading>
    </sensor>
</sensor_data>'''

# TODO: XML parsen

# TODO: Alle Sensor-IDs und Typen ausgeben

# TODO: Durchschnittswert pro Sensor berechnen (nur quality="valid")

# TODO: Nur valide Messungen anzeigen


### (c) XML-Datei lesen und analysieren - Labormessungen

Lies die Datei `../data/measurements.xml` und fuehre folgende Analysen durch:

- Zeige Metadaten der Messreihe (Projekt, Ort, Operator)
- Anzahl der Messwerte pro Sensor
- Berechne Minimum, Maximum und Durchschnitt pro Sensor
- Finde den hoechsten Temperaturwert mit Zeitstempel
- Liste alle Messungen vom 2025-11-02

In [ ]:
# Deine Loesung:
import xml.etree.ElementTree as ET

# TODO: XML-Datei laden mit ET.parse()

# TODO: Metadaten ausgeben

# TODO: Messwerte pro Sensor zaehlen

# TODO: Min, Max, Durchschnitt pro Sensor berechnen

# TODO: Hoechsten Temperaturwert finden

# TODO: Messungen vom 2025-11-02 filtern


### (d) XML-Datei modifizieren - Experimentdaten bearbeiten

Lade die Datei `../data/experiments.xml` und fuehre folgende Aenderungen durch:

- Setze den Status von Experiment "EXP-003" auf "abgeschlossen"
- Aktualisiere die Ergebnisse von "EXP-003" (Ausbeute: 89.3%, Reinheit: 97.5%)
- Fuege ein neues Experiment "EXP-005" hinzu (Temperatur: 45C, status: geplant)
- Entferne alle Experimente mit Status "fehlgeschlagen"
- Speichere das Ergebnis als `../data/experiments_updated.xml`

In [ ]:
# Deine Loesung:
import xml.etree.ElementTree as ET

# TODO: XML-Datei laden

# TODO: Status von EXP-003 aktualisieren

# TODO: Ergebnisse von EXP-003 aktualisieren

# TODO: Neues Experiment hinzufuegen

# TODO: Fehlgeschlagene Experimente entfernen

# TODO: Als neue Datei speichern


---
## 5 Loesungen


In [ ]:
# ====================================================================
# Musterloesung (a) - XML programmatisch erstellen - Messdaten-Struktur
# ====================================================================

import xml.etree.ElementTree as ET

# Root-Element mit Attributen erstellen
experiment = ET.Element("experiment")
experiment.set("id", "EXP-2025-042")
experiment.set("date", "2025-11-20")

# Metadata-Element mit Unterelementen
metadata = ET.SubElement(experiment, "metadata")
ET.SubElement(metadata, "researcher").text = "Dr. Anna Weber"
ET.SubElement(metadata, "location").text = "Labor B2, Raum 105"
ET.SubElement(metadata, "description").text = "Temperaturmessung waehrend Katalyse-Reaktion"

# Messung 1
m1 = ET.SubElement(experiment, "measurement")
m1.set("timestamp", "2025-11-20T09:00:00")
value1 = ET.SubElement(m1, "value")
value1.set("unit", "Celsius")
value1.text = "25.3"
ET.SubElement(m1, "quality").text = "valid"

# Messung 2
m2 = ET.SubElement(experiment, "measurement")
m2.set("timestamp", "2025-11-20T09:15:00")
value2 = ET.SubElement(m2, "value")
value2.set("unit", "Celsius")
value2.text = "47.8"
ET.SubElement(m2, "quality").text = "valid"

# Messung 3
m3 = ET.SubElement(experiment, "measurement")
m3.set("timestamp", "2025-11-20T09:30:00")
value3 = ET.SubElement(m3, "value")
value3.set("unit", "Celsius")
value3.text = "72.1"
ET.SubElement(m3, "quality").text = "valid"

# XML ausgeben
print("=== Experiment-Messdaten ===")
ET.dump(experiment)


In [ ]:
# ====================================================================
# Musterloesung (b) - XML-String parsen - Sensordaten auswerten
# ====================================================================

import xml.etree.ElementTree as ET

xml_string = '''<sensor_data station="Wetterstation-Nord">
    <sensor id="T01" type="temperature" unit="Celsius">
        <reading timestamp="08:00" quality="valid">18.5</reading>
        <reading timestamp="12:00" quality="valid">23.2</reading>
        <reading timestamp="16:00" quality="invalid">-999</reading>
        <reading timestamp="20:00" quality="valid">19.8</reading>
    </sensor>
    <sensor id="H01" type="humidity" unit="Prozent">
        <reading timestamp="08:00" quality="valid">65.2</reading>
        <reading timestamp="12:00" quality="valid">48.7</reading>
        <reading timestamp="16:00" quality="valid">52.1</reading>
        <reading timestamp="20:00" quality="valid">71.3</reading>
    </sensor>
</sensor_data>'''

# XML parsen
root = ET.fromstring(xml_string)
print(f"Station: {root.get('station')}")
print("=" * 50)

# Alle Sensor-IDs und Typen ausgeben
print("\n=== Sensoren ===")
for sensor in root.findall("sensor"):
    sensor_id = sensor.get("id")
    sensor_type = sensor.get("type")
    unit = sensor.get("unit")
    print(f"  {sensor_id}: {sensor_type} ({unit})")

# Durchschnittswert pro Sensor berechnen (nur valide Messungen)
print("\n=== Durchschnittswerte (nur valide) ===")
for sensor in root.findall("sensor"):
    sensor_id = sensor.get("id")
    unit = sensor.get("unit")
    values = []
    for reading in sensor.findall("reading"):
        if reading.get("quality") == "valid":
            values.append(float(reading.text))
    if values:
        avg = sum(values) / len(values)
        print(f"  {sensor_id}: {avg:.2f} {unit} (n={len(values)})")

# Nur valide Messungen anzeigen
print("\n=== Valide Messungen ===")
for sensor in root.findall("sensor"):
    sensor_id = sensor.get("id")
    for reading in sensor.findall("reading"):
        if reading.get("quality") == "valid":
            timestamp = reading.get("timestamp")
            value = reading.text
            print(f"  {sensor_id} @ {timestamp}: {value}")


In [ ]:
# ====================================================================
# Musterloesung (c) - XML-Datei lesen und analysieren - Labormessungen
# ====================================================================

import xml.etree.ElementTree as ET

# XML-Datei laden
tree = ET.parse("../data/measurements.xml")
root = tree.getroot()

# Metadaten ausgeben
print("=== Messreihen-Metadaten ===")
print(f"  Projekt: {root.get('project')}")
print(f"  ID: {root.get('id')}")
metadata = root.find("metadata")
print(f"  Ort: {metadata.find('location').text}")
print(f"  Operator: {metadata.find('operator').text}")
print(f"  Zeitraum: {metadata.find('start_date').text} bis {metadata.find('end_date').text}")

# Messwerte pro Sensor zaehlen
print("\n=== Messwerte pro Sensor ===")
for sensor in root.findall("sensor"):
    sensor_id = sensor.get("id")
    sensor_type = sensor.get("type")
    count = len(sensor.findall("reading"))
    print(f"  {sensor_id} ({sensor_type}): {count} Messwerte")

# Min, Max, Durchschnitt pro Sensor berechnen
print("\n=== Statistik pro Sensor ===")
for sensor in root.findall("sensor"):
    sensor_id = sensor.get("id")
    sensor_type = sensor.get("type")
    unit = sensor.get("unit")
    values = [float(r.text) for r in sensor.findall("reading")]
    print(f"  {sensor_type} ({sensor_id}):")
    print(f"    Min: {min(values):.1f} {unit}")
    print(f"    Max: {max(values):.1f} {unit}")
    print(f"    Avg: {sum(values)/len(values):.2f} {unit}")

# Hoechsten Temperaturwert finden
print("\n=== Hoechster Temperaturwert ===")
max_temp = float('-inf')
max_timestamp = None
for sensor in root.findall("sensor"):
    if sensor.get("type") == "temperature":
        for reading in sensor.findall("reading"):
            temp = float(reading.text)
            if temp > max_temp:
                max_temp = temp
                max_timestamp = reading.get("timestamp")
print(f"  {max_temp} Celsius @ {max_timestamp}")

# Messungen vom 2025-11-02 filtern
print("\n=== Messungen vom 2025-11-02 ===")
for sensor in root.findall("sensor"):
    sensor_type = sensor.get("type")
    unit = sensor.get("unit")
    for reading in sensor.findall("reading"):
        timestamp = reading.get("timestamp")
        if timestamp.startswith("2025-11-02"):
            time_part = timestamp.split("T")[1]
            print(f"  {time_part} - {sensor_type}: {reading.text} {unit}")


In [ ]:
# ====================================================================
# Musterloesung (d) - XML-Datei modifizieren - Experimentdaten bearbeiten
# ====================================================================

import xml.etree.ElementTree as ET

# XML-Datei laden
tree = ET.parse("../data/experiments.xml")
root = tree.getroot()

print(f"Labor: {root.get('name')}")
print("=== Vor den Aenderungen ===")
for exp in root.findall("experiment"):
    exp_id = exp.get("id")
    status = exp.get("status")
    title = exp.find("title").text
    print(f"  {exp_id}: {title} [{status}]")

# 1. Status von EXP-003 auf "abgeschlossen" setzen
for exp in root.findall("experiment"):
    if exp.get("id") == "EXP-003":
        exp.set("status", "abgeschlossen")
        print(f"\nStatus von EXP-003 aktualisiert")

# 2. Ergebnisse von EXP-003 aktualisieren
for exp in root.findall("experiment"):
    if exp.get("id") == "EXP-003":
        results = exp.find("results")
        for measurement in results.findall("measurement"):
            name = measurement.get("name")
            if name == "Ausbeute":
                measurement.set("value", "89.3")
            elif name == "Reinheit":
                measurement.set("value", "97.5")
        print("Ergebnisse von EXP-003 aktualisiert")

# 3. Neues Experiment EXP-005 hinzufuegen
new_exp = ET.SubElement(root, "experiment")
new_exp.set("id", "EXP-005")
new_exp.set("status", "geplant")
ET.SubElement(new_exp, "title").text = "Reaktionskinetik Versuch E"
ET.SubElement(new_exp, "date").text = "2025-10-25"
ET.SubElement(new_exp, "researcher").text = "K. Klein"
conditions = ET.SubElement(new_exp, "conditions")
temp = ET.SubElement(conditions, "temperature")
temp.set("unit", "Celsius")
temp.text = "45.0"
ET.SubElement(conditions, "pressure").text = "1.013"
ET.SubElement(conditions, "ph_value").text = "7.0"
results = ET.SubElement(new_exp, "results")
print("Neues Experiment EXP-005 hinzugefuegt")

# 4. Fehlgeschlagene Experimente entfernen
to_remove = []
for exp in root.findall("experiment"):
    if exp.get("status") == "fehlgeschlagen":
        to_remove.append(exp)
for exp in to_remove:
    root.remove(exp)
    print(f"Experiment {exp.get('id')} entfernt (fehlgeschlagen)")

print("\n=== Nach den Aenderungen ===")
for exp in root.findall("experiment"):
    exp_id = exp.get("id")
    status = exp.get("status")
    title = exp.find("title").text
    print(f"  {exp_id}: {title} [{status}]")

# 5. Als neue Datei speichern
tree.write("../data/experiments_updated.xml", encoding="utf-8", xml_declaration=True)
print("\nDatei gespeichert als: ../data/experiments_updated.xml")
